In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_date, col, lit

# Tạo Spark session
spark = SparkSession.builder.appName("ChicagoCrimeAnalysis").getOrCreate()

# Đọc dữ liệu từ file parquet
df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")

# Chuyển 'Date' sang timestamp
df = df.withColumn("DateTS", to_timestamp(col("Date"), "MM/dd/yyyy hh:mm:ss a"))

# Tạo biến ngày bắt đầu và kết thúc
start_date_str = '03/15/2018'
end_date_str = '03/31/2018'

# Lọc theo ngày
df_filtered = df.filter(
    (col("DateTS").cast("date") >= to_date(lit(start_date_str), "MM/dd/yyyy")) &
    (col("DateTS").cast("date") <= to_date(lit(end_date_str), "MM/dd/yyyy"))
)

# 👉 Thêm cột ngày đơn thuần (không có giờ)
df_filtered = df_filtered.withColumn("CrimeDate", to_date(col("DateTS")))

# 👉 Nhóm theo ngày và loại tội phạm
daily_stats = df_filtered.groupBy("CrimeDate", "PrimaryType").count().orderBy("CrimeDate", ascending=True)

# 👉 Ghi ra file CSV
daily_stats.coalesce(1).write.csv("C:/Users/ADMIN/output/ToiPhamTheoNgay", header=True, mode="overwrite")

print("✅ Đã ghi báo cáo theo ngày thành công.")


Số bản ghi sau khi lọc: 1
+-------------------+---------------+
|             DateTS|    PrimaryType|
+-------------------+---------------+
|2018-03-21 08:00:00|CRIMINAL DAMAGE|
+-------------------+---------------+

✅ Xuất file CSV thành công.
